good reference for stacking: https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/14335
        

### Loading needed libs

In [7]:
import numpy as np
import math
import pandas as pd

import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import *

from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import *
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pickle
from sklearn.svm import SVC
from xgboost import XGBClassifier
from time import time
from keras.utils.np_utils import to_categorical  
from sklearn.linear_model import *
import lightgbm as lgb
from sklearn.feature_selection import SelectKBest, chi2

### Finding the features

In [8]:
# make predictions for multinoulli NB
df = pd.read_csv('reddit_train.csv')
df2 = pd.read_csv('reddit_test.csv')
# df = df.sample(frac=1)
# df = df.iloc[:3000,:]

In [9]:
df['category_id'], mapping = df['subreddits'].factorize()

In [11]:
tfidf = TfidfVectorizer()
features_train = tfidf.fit_transform(df.comments)
features_test = tfidf.transform(df2.comments)
labels = df['category_id']

### Code for fine tuning and check which models are the best

In [5]:
# tfidf = TfidfVectorizer()
# features_main = tfidf.fit_transform(df.comments)
# features_test = tfidf.transform(df2.comments)
# labels = df.category_id
# plt.figure(figsize=(18,6),dpi=100)
# for k in [10000, 20000, 30000]:

#     features = SelectKBest(chi2, k=k).fit_transform(features_main, labels)

#     test_acc = []
#     vals = [0.1]
#     for i in vals:
#         gNB = RidgeClassifier()
#         gNB.fit(features[:50000,:],labels[:50000])
#         test_acc.append(np.mean(gNB.predict(features[50000:,:])==labels[50000:]))
# #         print(i, np.mean(gNB.predict(features[50000:,:])==labels[50000:]))

#     plt.plot(vals, test_acc, '.-', label=str(k))
# plt.legend()
# plt.show()

In [6]:
# tfidf = TfidfVectorizer()
# features_main = tfidf.fit_transform(df.comments)
# features_test = tfidf.transform(df2.comments)
# labels = df.category_id
# plt.figure(figsize=(18,6),dpi=100)
# for k in [10000, 20000, 30000]:

#     features = SelectKBest(chi2, k=k).fit_transform(features_main, labels)

#     test_acc = []
#     vals = [0.1]
#     for i in vals:
#         gNB = RidgeClassifier()
#         gNB.fit(features[:50000,:],labels[:50000])
#         test_acc.append(np.mean(gNB.predict(features[50000:,:])==labels[50000:]))
# #         print(i, np.mean(gNB.predict(features[50000:,:])==labels[50000:]))

#     plt.plot(vals, test_acc, '.-', label=str(k))
# plt.legend()
# plt.show()

### Extracting features from a sklearn model

In [12]:
def getFeatures(classifier, features_train, features_test, labels, do_cv=True):
    if(do_cv):
        cv_acc = cross_val_score(classifier, features_train, labels, scoring='accuracy', cv=5)
        print(' cross validation: ', cv_acc)
    classifier.fit(features_train, labels)
    f1 = classifier.predict_proba(features_train)
    f2 = classifier.predict_proba(features_test)
    print(' training accuracy: ', np.mean(classifier.predict(features_train)==labels))
    return f1, f2

def selectFeatures(features_train, labels, features_test, k=15000):
    selector = SelectKBest(chi2, k=k)
    Xtrain = selector.fit_transform(features_train, labels)
    Xtest = selector.transform(features_test)
    return Xtrain, Xtest

def getFeaturesOneHot(classifier, features_train, features_test, labels, do_cv=True):
    if(do_cv):
        cv_acc = cross_val_score(classifier, features_train, labels, scoring='accuracy', cv=5)
        print(' cross validation: ', cv_acc)
    classifier.fit(features_train, labels)
    f1 = classifier.predict(features_train)
    f2 = classifier.predict(features_test)
    print(' training accuracy: ', np.mean(classifier.predict(features_train)==labels))
    return to_categorical(f1, num_classes=20), to_categorical(f2, num_classes=20)

In [13]:
tmp_tr, tmp_te = selectFeatures(features_train, labels, features_test, k=16000)
m3_train, m3_test = getFeatures(MultinomialNB(0.05), tmp_tr, tmp_te, labels)
tmp_tr, tmp_te = selectFeatures(features_train, labels, features_test, k=11000)
m4_train, m4_test = getFeaturesOneHot(LinearSVC(C=0.5), tmp_tr, tmp_te, labels)
# m9_train, m9_test = getFeaturesOneHot(RidgeClassifier())

 cross validation:  [0.5985     0.60178571 0.59757143 0.59671429 0.60064286]
 training accuracy:  0.7264285714285714
 cross validation:  [0.5835     0.59071429 0.5845     0.58192857 0.58378571]
 training accuracy:  0.7068285714285715


### Extracting features from pytorch models (saved)

In [14]:
# load predictions
def pickle2np(filename="preds/bert-preds-test.p"):
    tmp = pickle.load( open( filename, "rb" ) )
    preds = list()
    for i in range(len(tmp)):
        for j in range(len(tmp[i])):
            preds.append(list(tmp[i][j]))
            
    return np.array(preds)

m1_train = pickle2np("preds/bert-preds-train.p")
m1_test = pickle2np("preds/bert-preds-test.p")
m2_train = pickle2np("preds/xlnet-preds-train.p")
m2_test = pickle2np("preds/xlnet-preds-test.p")

In [15]:
# m3_train = m3_train[:3000, :]
# m4_train = m4_train[:3000, :]

In [16]:
# stacking the features
features_tr = np.concatenate((m1_train, m2_train, m3_train, m4_train),axis=1)
features_te = np.concatenate((m1_test, m2_test, m3_test, m4_test),axis=1)
labels = df.category_id
print(features_tr.shape, features_te.shape, labels.shape)

(70000, 80) (30000, 80) (70000,)


In [27]:
# uncomment if you want to work on a smaller sample
samples = np.random.randint(70000,size=(1000))
features = features[samples,:]
labels = df.category_id[samples]
features.shape

IndexError: index 24413 is out of bounds for axis 0 with size 3000

In [72]:
y_pred = gbm.predict(features_tr[:60000], num_iteration=gbm.best_iteration)
print('train_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[:60000]))

train_acc:  0.83285


In [69]:
params = {
 'task': 'train'
 , 'boosting_type': 'gbdt'
 , 'objective': 'multiclass'
 , 'num_class': 20
 , 'metric': 'multi_logloss'
 , 'min_data': 1
 , 'verbose': -1
}

lgb_train = lgb.Dataset(features_tr[:60000,:], labels[:60000])
lgb_eval = lgb.Dataset(features_tr[60000:,:], labels[60000:], reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

y_pred = gbm.predict(features_tr[:60000], num_iteration=gbm.best_iteration)
print('train_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[:60000]))
y_pred = gbm.predict(features_tr[60000:], num_iteration=gbm.best_iteration)
print('test_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[60000:]))

[1]	valid_0's multi_logloss: 2.4001
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 2.14075
[3]	valid_0's multi_logloss: 1.96026
[4]	valid_0's multi_logloss: 1.82044
[5]	valid_0's multi_logloss: 1.70722
[6]	valid_0's multi_logloss: 1.61284
[7]	valid_0's multi_logloss: 1.5321
[8]	valid_0's multi_logloss: 1.46263
[9]	valid_0's multi_logloss: 1.40261
[10]	valid_0's multi_logloss: 1.34992
[11]	valid_0's multi_logloss: 1.30321
[12]	valid_0's multi_logloss: 1.26156
[13]	valid_0's multi_logloss: 1.2246
[14]	valid_0's multi_logloss: 1.19129
[15]	valid_0's multi_logloss: 1.16145
[16]	valid_0's multi_logloss: 1.13441
[17]	valid_0's multi_logloss: 1.1098
[18]	valid_0's multi_logloss: 1.08802
[19]	valid_0's multi_logloss: 1.06878
[20]	valid_0's multi_logloss: 1.0512
[21]	valid_0's multi_logloss: 1.03511
[22]	valid_0's multi_logloss: 1.02059
[23]	valid_0's multi_logloss: 1.00745
[24]	valid_0's multi_logloss: 0.995536
[25]	valid_0's multi_logloss: 0.984328
[2

In [74]:
y_pred = reg.predict(features_tr[:60000])
print('train_acc: ', np.mean(y_pred==labels[:60000]))
y_pred = reg.predict(features_tr[60000:])
print('test_acc: ', np.mean(y_pred==labels[60000:]))

train_acc:  0.7565666666666667
test_acc:  0.7423


In [73]:
print(features_tr.shape,labels.shape)
reg = XGBClassifier()
start_time = time()
reg.fit(features_tr[:60000],labels[:60000])
print(time()-start_time)
y_pred = reg.predict(features_tr[:60000])
print('train_acc: ', np.mean(y_pred==labels[:60000]))
y_pred = reg.predict(features_tr[60000:])
print('test_acc: ', np.mean(y_pred==labels[60000:]))

(70000, 80) (70000,)
483.2753665447235
train_acc:  7105      True
15578    False
44994     True
34706    False
30169     True
         ...  
68895     True
45005     True
28733    False
29234    False
16829     True
Name: category_id, Length: 60000, dtype: bool
test_acc:  44192     True
6004      True
5089     False
68149     True
59670    False
         ...  
56240    False
6718      True
36722     True
11757     True
15685     True
Name: category_id, Length: 10000, dtype: bool


### fine tuning the meta learner

In [29]:
for eta in [0.01, 0.1, 0.2, 0.3, 1]:
    gNB = XGBClassifier(learning_rate=eta)
    gNB.fit(features[:900,:],labels[:900])
    print(eta, np.mean(gNB.predict(features[900:,:])==labels[900:]))

0.01 0.89
0.1 0.9
0.2 0.88
0.3 0.88
1 0.85


### fit classifier on full dataset

In [17]:
print(features_tr.shape,labels.shape)
reg = XGBClassifier()
start_time = time()
reg.fit(features_tr,labels)
print(time()-start_time)
y_pred = reg.predict(features_tr)
print('train_acc: ', np.mean(y_pred==labels))

(70000, 80) (70000,)
565.3684413433075
train_acc:  0.8282285714285714


In [18]:
params = {
 'task': 'train'
 , 'boosting_type': 'gbdt'
 , 'objective': 'multiclass'
 , 'num_class': 20
 , 'metric': 'multi_logloss'
 , 'min_data': 1
 , 'verbose': -1
}

lgb_train = lgb.Dataset(features_tr[:60000,:], labels[:60000])
lgb_eval = lgb.Dataset(features_tr[60000:,:], labels[60000:], reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

y_pred = gbm.predict(features_tr[:60000], num_iteration=gbm.best_iteration)
print('train_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[:60000]))
y_pred = gbm.predict(features_tr[60000:], num_iteration=gbm.best_iteration)
print('test_acc: ', np.mean(np.argmax(y_pred, axis=1)==labels[60000:]))

[1]	valid_0's multi_logloss: 2.35294
Training until validation scores don't improve for 5 rounds
[2]	valid_0's multi_logloss: 2.07322
[3]	valid_0's multi_logloss: 1.87815
[4]	valid_0's multi_logloss: 1.72695
[5]	valid_0's multi_logloss: 1.60418
[6]	valid_0's multi_logloss: 1.50152
[7]	valid_0's multi_logloss: 1.4142
[8]	valid_0's multi_logloss: 1.33899
[9]	valid_0's multi_logloss: 1.2723
[10]	valid_0's multi_logloss: 1.21348
[11]	valid_0's multi_logloss: 1.1612
[12]	valid_0's multi_logloss: 1.11491
[13]	valid_0's multi_logloss: 1.07342
[14]	valid_0's multi_logloss: 1.0361
[15]	valid_0's multi_logloss: 1.00265
[16]	valid_0's multi_logloss: 0.9719
[17]	valid_0's multi_logloss: 0.944219
[18]	valid_0's multi_logloss: 0.919167
[19]	valid_0's multi_logloss: 0.896006
[20]	valid_0's multi_logloss: 0.875473
[21]	valid_0's multi_logloss: 0.856097
[22]	valid_0's multi_logloss: 0.839466
[23]	valid_0's multi_logloss: 0.823055
[24]	valid_0's multi_logloss: 0.808562
[25]	valid_0's multi_logloss: 0.79

In [30]:
# Combined train accuracy
preds = np.argmax(reg.predict_proba(features_tr) + gbm.predict(features_tr, num_iteration=gbm.best_iteration), axis=1)
np.mean(preds==labels)

0.8646857142857143

In [31]:
preds = np.argmax(reg.predict_proba(features_te) + gbm.predict(features_te, num_iteration=gbm.best_iteration), axis=1)

In [32]:
test_preds = pd.DataFrame()
test_preds['Id'] = df2['id']
test_preds['Category'] = mapping[preds]


In [33]:
test_preds.to_csv("test.csv", index=False)
from IPython.display import FileLink, FileLinks
FileLink('test.csv')

/network/home/penmetss/comp551/test.csv